# List intersection algorithms

In this notebook, you can experiment with two algorithms (the naive one and the one using galloping search) to perform list intersection, which is an essential operator in supporting conjunction keyword queries.


In [76]:
## define the function p() that outputs debugging messages if DEBUG flag is True. 

#DEBUG = True
DEBUG = False

def p(msg):
    if DEBUG:
        print(".. {}".format(msg)) 

## The InvertedList class

We implement this class which encapsulates all the important *operations* one can perform on an inverted list. 

In [77]:
from pprint import pprint
from bisect import *
from math import *
import sys


sys.setrecursionlimit(1000000000) 
## You may find many similarity between this class and the file operation (in C/Java). 
class InvertedList:
    def __init__(self, l):
        self.data = l[:] # make a copy
        self.cur = 0     # the cursor 

    def get_list(self):
        return self.data
 
    def eol(self):
        # we use cur == len(list) to indicate EOL
        return False if self.cur < len(self.data) else True
    
    def next(self, val = 1):
        # does not allow cur to be out-of-range, but use len(list) to indicate EOL
        self.cur = min(self.cur + val, len(self.data)) 
            
    def elem(self):
        if self.eol():
            return None
        else: 
            return self.data[self.cur]

    def peek(self, pos):
        # look at the element under the current cursor, but does not advance the cursor. 
        if pos < len(self.data):
            return self.data[pos]
        else:
            return None
        
    def reset(self):
        self.cur = 0
    
    def dump(self):
        print("* {} : {} (cur = {})".format(self.data[:self.cur], self.data[self.cur:], self.cur))
        
    def skip_to(self, val):
        ##!! This is a naive and inefficient implementation. 
        ##!! Once you have implemented gallop_to(), you can just call it here instead.
        # move the cursor to the first element no smaller than `val`

        while not self.eol() and self.elem() < val:
            self.next()               
            
    def binary_search_recursive(self, val, list_in, start, end):
        mid = floor((start + end)/2)
        if self.data[mid]==val:
            return mid
        else:
            if self.data[mid]<val:
                return self.binary_search(val, list_in, mid+1, end)
            else:
                return self.binary_search(val, list_in, start, mid-1)
        
    def binary_search_non_recursive(self, val, list_in, start, end):
        
        low = start 
        high = end
        mid = 0
        print("start is %d, end is %d at the begining"%(start,end))
        while low<high:
            mid = floor((low+high)/2)
            print("mid in while is %d"%mid)
            if self.data[mid]==val:
                return mid;
            elif self.data[mid]>val:
                high = mid - 1
            else:
                low = mid + 1
        print("mid out of while is %d"%mid)
        return ceil((low+high)/2)
        
        
    def gallop_to(self, val):
        ##!! Not implemented yet. 
        #pass
        step_len = 1
        old_cur = 0
        
        try:
            while self.elem() < val:
                print("self elem() is %d"%self.elem())
                old_cur = self.cur
                self.next(step_len)
                step_len = step_len*2
        except:
            
            print("none value at current position")
          
            
        
            
        end = self.cur
        start = old_cur
        
        print("start is %d and end is %d"%(start, end))
        
        
        
        tmp_cur = self.binary_search_non_recursive(val, self.data, start, end)
        print("tmp cur is %d"%tmp_cur)
        self.cur = min(tmp_cur, len(self.data)) 
        
        

                        





### Test the class

In [78]:
a = InvertedList([2, 4, 6, 8, 10, 12, 14, 16, 18])
b = InvertedList([1, 2, 4, 8, 16, 32])

while not a.eol():
    e = a.elem()
    a.next()
    print(e)

a.dump()

2
4
6
8
10
12
14
16
18
* [2, 4, 6, 8, 10, 12, 14, 16, 18] : [] (cur = 9)


If you have implemented the `gallop_to()` correctly, you should pass the following test. You may change the call to `skip_to()` for now to see how it should work. 

In [79]:
def test_gallop(l, val):
    print("=> gallop_to({})".format(val))
    l.reset()
    l.gallop_to(val)
    #l.skip_to(val)
    l.dump()

In [80]:
test_a = [val - 1 for val in a.get_list()]
test_a.append(a.get_list()[-1] + 100)

DEBUG = False # to show the overview
a.dump()
for t_a in test_a:
    test_gallop(a, t_a)

* [2, 4, 6, 8, 10, 12, 14, 16, 18] : [] (cur = 9)
=> gallop_to(1)
start is 0 and end is 0
start is 0, end is 0 at the begining
mid out of while is 0
tmp cur is 0
* [] : [2, 4, 6, 8, 10, 12, 14, 16, 18] (cur = 0)
=> gallop_to(3)
self elem() is 2
start is 0 and end is 1
start is 0, end is 1 at the begining
mid in while is 0
mid out of while is 0
tmp cur is 1
* [2] : [4, 6, 8, 10, 12, 14, 16, 18] (cur = 1)
=> gallop_to(5)
self elem() is 2
self elem() is 4
start is 1 and end is 3
start is 1, end is 3 at the begining
mid in while is 2
mid out of while is 2
tmp cur is 1
* [2] : [4, 6, 8, 10, 12, 14, 16, 18] (cur = 1)
=> gallop_to(7)
self elem() is 2
self elem() is 4
start is 1 and end is 3
start is 1, end is 3 at the begining
mid in while is 2
mid out of while is 2
tmp cur is 3
* [2, 4, 6] : [8, 10, 12, 14, 16, 18] (cur = 3)
=> gallop_to(9)
self elem() is 2
self elem() is 4
self elem() is 8
start is 3 and end is 7
start is 3, end is 7 at the begining
mid in while is 5
mid in while is 3
mid o

In [81]:
DEBUG = True # to show the details. Your detailed debugging info may vary. 
test_gallop(a, test_a[-2])

=> gallop_to(17)
self elem() is 2
self elem() is 4
self elem() is 8
self elem() is 16
none value at current position
start is 7 and end is 9
start is 7, end is 9 at the begining
mid in while is 8
mid out of while is 8
tmp cur is 7
* [2, 4, 6, 8, 10, 12, 14] : [16, 18] (cur = 7)


As you can see above, the galloping search algorithm quickly skip over the list to reposition the cursor. In the additional note, it is proved that its complexity is $O(\log x)$, where $x$ is the difference between the correct cursor position and the initial cursor position. 

## The Naive Intersection Algorithm

In [82]:
def intersection_naive(a, b):
    # just in case these lists have been traversed.
    a.reset()
    b.reset()
    
    ret = []
    while not a.eol() and not b.eol():
        if a.elem() < b.elem():
            p("move a")
            a.next()
        elif a.elem() > b.elem():
            p("move b")
            b.next()
        else:
            p("found one")
            ret.append(a.elem())
            a.next()
            b.next()
    return ret
        

Test

In [83]:
intersection_naive(a, b)

.. move b
.. found one
.. found one
.. move a
.. found one
.. move a
.. move a
.. move a
.. found one
.. move a


[2, 4, 8, 16]

## Intersection Algorithm using `gallop_to()`

Here, we call `gallop_to` explicitly to show some detailed info if you have it implemented correctly. You can change the calls to `skip_to` in a more general setting. 

In [84]:
def intersection_galloping(a, b):
    # just in case these lists have been traversed.
    a.reset()
    b.reset()
    
    ret = []
    while not a.eol() and not b.eol():
        if a.elem() == b.elem():
            p("found one")
            ret.append(a.elem())
            a.next()
        else:
            if a.elem() < b.elem():
                a.gallop_to(b.elem())
            else:
                b.gallop_to(a.elem())
    # end_while
    return ret
        

In [85]:
intersection_galloping(a, b)

self elem() is 1
start is 0 and end is 1
start is 0, end is 1 at the begining
mid in while is 0
mid out of while is 0
tmp cur is 1
.. found one
self elem() is 2
start is 1 and end is 2
start is 1, end is 2 at the begining
mid in while is 1
mid out of while is 1
tmp cur is 2
.. found one
self elem() is 4
start is 2 and end is 3
start is 2, end is 3 at the begining
mid in while is 2
mid out of while is 2
tmp cur is 3
self elem() is 6
start is 2 and end is 3
start is 2, end is 3 at the begining
mid in while is 2
mid out of while is 2
tmp cur is 3
.. found one
self elem() is 8
start is 3 and end is 4
start is 3, end is 4 at the begining
mid in while is 3
mid out of while is 3
tmp cur is 4
self elem() is 10
self elem() is 12
start is 5 and end is 7
start is 5, end is 7 at the begining
mid in while is 6
mid out of while is 6
tmp cur is 7
.. found one
self elem() is 16
start is 4 and end is 5
start is 4, end is 5 at the begining
mid in while is 4
mid out of while is 4
tmp cur is 5
self elem()

[2, 4, 8, 16]

## Exercise

You need to correctly implement `gallop_to` using the galloping search algorithm, and then play with the intersection algorithm to appreciate how efficient it works in action. Note that the implementation is not hard, but you do need to take care of a few corner cases. That's why the `test_gallop()` evaluates almost all possible situation. 